# 基准

In [1]:
from tvm import relay
from tvm.ir import IRModule
from tvm.relay import testing

def get_network(name, batch_size, dtype="float32"):
    """Get the symbol definition and random weight of a network

    Parameters
    ----------
    name: str
        The name of the network, can be 'resnet-18', 'resnet-50', 'vgg-16', 'inception_v3', 'mobilenet', ...
    batch_size: int
        batch size
    dtype: str
        Data type

    Returns
    -------
    net: tvm.IRModule
        The relay function of network definition
    params: dict
        The random parameters for benchmark
    input_shape: tuple
        The shape of input tensor
    output_shape: tuple
        The shape of output tensor
    """
    input_shape = (batch_size, 3, 224, 224)
    output_shape = (batch_size, 1000)

    if name == "mobilenet":
        net, params = testing.mobilenet.get_workload(batch_size=batch_size, dtype=dtype)
    elif name == "inception_v3":
        input_shape = (batch_size, 3, 299, 299)
        net, params = testing.inception_v3.get_workload(batch_size=batch_size, dtype=dtype)
    elif "resnet" in name:
        n_layer = int(name.split("-")[1])
        net, params = testing.resnet.get_workload(
            num_layers=n_layer, batch_size=batch_size, dtype=dtype
        )
    elif "vgg" in name:
        n_layer = int(name.split("-")[1])
        net, params = testing.vgg.get_workload(
            num_layers=n_layer, batch_size=batch_size, dtype=dtype
        )
    elif "densenet" in name:
        n_layer = int(name.split("-")[1])
        net, params = testing.densenet.get_workload(
            densenet_size=n_layer, batch_size=batch_size, dtype=dtype
        )
    elif "squeezenet" in name:
        version = name.split("_v")[1]
        net, params = testing.squeezenet.get_workload(
            batch_size=batch_size, version=version, dtype=dtype
        )
    elif name == "mxnet":
        # an example for mxnet model
        from mxnet.gluon.model_zoo.vision import get_model

        block = get_model("resnet18_v1", pretrained=True)
        net, params = relay.frontend.from_mxnet(block, shape={"data": input_shape}, dtype=dtype)
        net = net["main"]
        net = relay.Function(
            net.params, relay.nn.softmax(net.body), None, net.type_params, net.attrs
        )
        net = IRModule.from_expr(net)
    else:
        raise ValueError("Unsupported network: " + name)
    return net, params, input_shape, output_shape

In [3]:
import tvm

host = "127.0.0.1"
port = 9190
rpc_key = "mate10pro"
repeat = 10
model = "mate10pro"
dtype = "float32"
network = None
if network is None:
    networks = ["squeezenet_v1.1", "mobilenet", "resnet-18", "vgg-16"]
else:
    networks = [network]

target = tvm.target.arm_cpu(model=model)
target_host = None

In [4]:
import numpy as np

import tvm
from tvm import te
from tvm.contrib.utils import tempdir
import tvm.contrib.graph_executor as runtime
from tvm import relay

def evaluate_network(network, target, target_host, repeat):
    # connect to remote device
    tracker = tvm.rpc.connect_tracker(host, port)
    remote = tracker.request(rpc_key)

    # print_progress(network)
    net, params, input_shape, output_shape = get_network(network, batch_size=1)

    # print_progress("%-20s building..." % network)
    with tvm.transform.PassContext(opt_level=3):
        lib = relay.build(net, target=tvm.target.Target(target, host=target_host), params=params)

    tmp = tempdir()
    if "android" in str(target):
        from tvm.contrib import ndk

        filename = "%s.so" % network
        lib.export_library(tmp.relpath(filename), ndk.create_shared)
    else:
        filename = "%s.tar" % network
        lib.export_library(tmp.relpath(filename))

    # upload library and params
    # print_progress("%-20s uploading..." % network)
    dev = remote.device(str(target), 0)
    remote.upload(tmp.relpath(filename))

    rlib = remote.load_module(filename)
    module = runtime.GraphModule(rlib["default"](dev))
    data_tvm = tvm.nd.array((np.random.uniform(size=input_shape)).astype(dtype))
    module.set_input("data", data_tvm)

    # evaluate
    # print_progress("%-20s evaluating..." % network)
    ftimer = module.module.time_evaluator("run", dev, number=1, repeat=repeat)
    prof_res = np.array(ftimer().results) * 1000  # multiply 1000 for converting to millisecond
    print(
        "%-20s %-19s (%s)" % (network, "%.2f ms" % np.mean(prof_res), "%.2f ms" % np.std(prof_res))
    )

In [5]:
for network in networks:
    evaluate_network(network, target, target_host, repeat)

KeyboardInterrupt: 